<a href="https://colab.research.google.com/github/Samiraa-Alipour/Sales_Prediction/blob/main/Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BigMart Sales Prediction
##Introduction
This is a Machine Learning project on the BigMart Sales Dataset. It's a Regression Task. The purpose of this project is to:
- Understand the Dataset & cleanup
- Build Regression models to predict the sales of the products
- Evaluate the models & compare their respective scores like R2, RMSE, etc.


This project is sequenced in the following steps:
- step1: Problem Statement
- step2: BigMart Dataset Description
- step3: Data Exploration
- step4: Data Preprocessing
- step5: Feature Selection
- step6: Model Selection
- step7: Training and Evaluation
- step8: Cross Validation
- step9: Interpretation and Insight

#Step1: Problem Statement
Sales prediction is an important aspect of different companies. It allows companies to efficiently allocate resources, to estimate achievable sales revenue and to plan a better strategy for future growth of the company.

**The aim** is to build *some predictive models* to predict *the sales of each product at a particular store* and compare their scores.
Using these models, BigMart will try to understand **the properties of products** and **stores** which play a key role in increasing sales.

This is **a supervised machine learning problem** with a target label as (Item_Outlet_Sales). Also since we are expected to predict the sales price for a given product, it becomes **a regression task**.

#step2:   BigMart Dataset Description

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, 12 attributes of each product and store have been defined, as below mentioned:
1.   **Item_Identifier** ---- Unique product ID
2.   **Item_Weight** ---- Weight of product
3.   **Item_Fat_Content** ---- Whether the product is low fat or not
4.   **Item_Visibility** ---- The % of the total display area of all products in a store allocated to the particular product
5.   **Item_Type** ---- The category to which the product belongs
6.   **Item_MRP** ---- Maximum Retail Price (list price) of the product
7.   **Outlet_Identifier** ---- Unique store ID
8.   **Outlet_Establishment_Year** ---- The year in which the store was established
9.   **Outlet_Size** ---- The size of the store in terms of ground area covered
10.  **Outlet_Location_Type**---- The type of city in which the store is located
11.  **Outlet_Type** ---- Whether the outlet is just a grocery store or some sort of supermarket
12.  **Item_Outlet_Sales** ---- sales of the product in t particular store. This is **the outcome variable to be predicted.**

This Dataset can be downloaded from: [kaggle](https://www.kaggle.com/datasets/brijbhushannanda1979/bigmart-sales-data)

Kaggle have both train.csv(8523,12) & test.csv(5681,11) sets, but we don't have the outcome column of test and we can't evaluate the model, so we just used train set.

In [ ]:
# Download DataSet
!gdown https://drive.google.com/uc?id=1YZyU9Fb8GQ9m9peWiHgr-WJYZNWJcz47

Downloading...
From: https://drive.google.com/uc?id=1YZyU9Fb8GQ9m9peWiHgr-WJYZNWJcz47
To: /content/BigMart_Train.csv
100% 870k/870k [00:00<00:00, 97.3MB/s]


In [ ]:
#import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection  import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics


In [ ]:
#Loading the Dataset
data = pd.read_csv('BigMart_Train.csv')

#check the 5 sample rows of dataset
data.sample(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
5845,NCQ02,12.6,Low Fat,0.007468,Household,186.9556,OUT049,1999,Medium,Tier 1,Supermarket Type1,3379.6008
5921,FDT20,10.5,Low Fat,0.041361,Fruits and Vegetables,39.5164,OUT013,1987,High,Tier 3,Supermarket Type1,926.7936
2567,FDM56,NaN,low fat,0.069852,Fruits and Vegetables,110.9912,OUT027,1985,Medium,Tier 3,Supermarket Type3,1310.2944
7605,FDD53,16.2,Low Fat,0.044473,Frozen Foods,43.3454,OUT017,2007,NaN,Tier 2,Supermarket Type1,922.7988
7663,FDI32,NaN,Low Fat,0.173529,Fruits and Vegetables,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,4376.9692


In [ ]:
# Dimension of the dataset
data.shape

(8523, 12)

In [ ]:
#name of columns
data.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

#step3: Data Exploration

As below, we noticed that:
- we have both **numerical** and **categorical** values which their datatypes are: float64, int64 and object
- **'Item_weight'** and **'Outlet_size'** have **missing values**
-  the **min** value of **'Item_Visibility'** is **zero**, it's impossible because each product takes some space
- dataset is about 1559 unique products in 10 unique outlet
- Item_Type contains 16 unique values
- *Two types* of **Item_Fat_Content** are there but some of them are misspelled as (*'low fat', 'LF'*) instead of **'Low Fat'** and *'regular'* instead of **'Regular'**
-

In [ ]:
#some information about the attributes(datatypes & null values)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
#Check for Null values
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
#Check statistical information of numerical values

numerical_features = data.select_dtypes(include=[np.number])
data.describe(include=[np.number])

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
#Check statistical information of categorical values

categorial_features = data.select_dtypes(include=object)
data.describe(include=object)

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


In [ ]:
#Check Unique Values of categorical attributes and their counts

for cat_col in categorial_features:
  if len(data[cat_col].unique()) <20:
    print("\n\n===> Unique Values & it's count of '{}' : \n{}".format(cat_col, data[cat_col].value_counts()))
  else:
    print("\n\n===> Unique Values of '{}' : \n{}".format(cat_col, data[cat_col].unique()))



===> Unique Values of 'Item_Identifier' : 
['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']


===> Unique Values & it's count of 'Item_Fat_Content' : 
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


===> Unique Values & it's count of 'Item_Type' : 
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


===> Unique Values & it's count of 'Outlet_Identifier' : 
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT01


#step4: Data Preprocessing
In thi step....... Data Preprocessing(Missing values, Outliers, Normalization, Encoding categorical data)

In [ ]:

#Missing VALUE

mean_Wper_items = pd.pivot_table(data, columns='Item_Identifier', values='Item_Weight', aggfunc=np.mean, fill_value=data['Item_Weight'].mean() , dropna=False)
missing_values = data['Item_Weight'].isnull()

data.loc[missing_values, 'Item_Weight'] = data.loc[missing_values, 'Item_Identifier'].apply(lambda x: mean_Wper_items[x].Item_Weight)



#step5: Feature Selection

(Correlation Analysis, Feature importance, using domain knowledge)

# step6: Model Selection


# step7: Training and Evaluation


#step8: Cross Validation


#step9: Interpretation and Insight